In [36]:
import pandas as pd
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
from torch.utils.data import DataLoader
from torch.utils.data import Dataset 


In [ ]:
#生成训练集
class Train_Loader(Dataset):
    def __init__(
            self, 
            data_dir, 
    ):
        self.data_dir = data_dir
        data = pd.read_excel(self.data_dir, skiprows=0)
        #将数据类型转换为float
        data = data.astype('float32')
        #缺失值使用上一行和下一行同一列的平均值填充
        data = data.fillna(data.mean())
        #对目标列(除最后三列)归一化
        scaler = MinMaxScaler(feature_range=(0, 1))
        data.iloc[:, :-3] = scaler.fit_transform(data.iloc[:, :-3])
        self.data = data.values
        self.scaler = scaler

    def __getitem__(self, i):
        features = self.data[i, :-3]
        #扩充为二维
        features = np.expand_dims(features, axis=0)

        targets = self.data[i, -3]
        #扩充为二维
        targets = np.array([[targets]])
        
        return features, targets
        
    def __len__(self):
        return len(self.data)
    

#生成测试集
class Test_Loader(Dataset):
    def __init__(
            self, 
            data_dir, 
            scaler,
    ):
        self.data_dir = data_dir
        data = pd.read_excel(self.data_dir, skiprows=0)
        #将数据类型转换为float
        data = data.astype('float32')
        #缺失值使用上一行和下一行同一列的平均值填充
        data = data.fillna(data.mean())
        #对目标列(除最后三列)归一化
        data.iloc[:, :-3] = scaler.transform(data.iloc[:, :-3])
        self.data = data.values

    def __getitem__(self, i):
        features = self.data[i, :-3]
        #扩充为二维
        features = np.expand_dims(features, axis=0)

        targets = self.data[i, -3]
        #扩充为二维
        targets = np.array([[targets]])
        return features, targets
        
    def __len__(self):
        return len(self.data)

In [ ]:
train_data_dir="data/train_data.xlsx"
train_data_loader = Train_Loader(train_data_dir)

In [ ]:
#查看feature，target数据形状
features, targets = train_data_loader[0]
print(features.shape)
print(targets.shape)
#查看train_data_loader数据长度
print(len(train_data_loader))

In [ ]:
batchsize=64
#创建数据加载器
train_data = DataLoader(train_data_loader, batch_size=batchsize, shuffle=True)
#查看数据加载器，数据形状
for features, targets in train_data:
    print(features.shape)
    print(targets.shape)
    break
print(len(train_data))

In [ ]:
# from nets.CNN_BIGRU_Attention import CNN_BiGRU_Attention
from nets.CNN_Transformer import CNN_Transformer
n_future = 30  # Example value, adjust as needed
n_class = 1   # Example value, adjust as needed
model = CNN_Transformer(n_future, n_class)

#定义损失函数
criterion = nn.MSELoss()
#定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)
# model_dir="model/CNN_BiGRU_Attention"
model_dir="model/CNN_Transformer_YS"


In [ ]:
# model=torch.load("model/CNN_Transformer247.41838136212579.pth")


In [ ]:
#计算验证集的MSE
valid_data_dir="data/valid_data.xlsx"
valid_data_loader = Test_Loader(valid_data_dir, train_data_loader.scaler)
valid_data = DataLoader(valid_data_loader, batch_size=batchsize, shuffle=False)
print(len(valid_data))


In [ ]:
def valid(valid_model, valid_data, criterion):
    # valid_model.eval()
    valid_loss = 0
    for features, targets in valid_data:
        features = features.cuda()
        targets = targets.cuda()
        valid_model = valid_model.cuda()
        outputs = valid_model(features)
        loss = criterion(outputs, targets)
        valid_loss = valid_loss+loss.item()
    # print('valid_loss:', valid_loss / len(valid_data))
    return valid_loss / len(valid_data)

In [ ]:
#直接训练模型，根据验证集MSE保存最优模型，超过50伦验证集MSE没有下降则修改学习率
min_valid_loss = float('inf')
no_improve = 0
for epoch in range(1000):
    model.train()
    train_loss = 0
    for features, targets in train_data:
        features = features.cuda()
        targets = targets.cuda()
        optimizer.zero_grad()
        model = model.cuda()
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss = train_loss+loss.item()
    valid_loss = valid(model, valid_data, criterion)
    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        torch.save(model, model_dir + str(min_valid_loss) + '.pth')
        no_improve = 0
    else:
        no_improve += 1
    if no_improve > 50:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
            print('Learning rate has been changed to: {}'.format(param_group['lr']))
        no_improve = 0
    print('Epoch: {}, Train Loss: {:.4f}, Valid Loss: {:.4f}'.format(epoch, train_loss / len(train_data), valid_loss))
    #将打印的训练集MSE和验证集MSE保存到txt文件
    with open(model_dir + 'loss.txt', 'a') as f:
        f.write('Epoch: {}, Train Loss: {:.4f}, Valid Loss: {:.4f}\n'.format(epoch, train_loss / len(train_data), valid_loss))
    if optimizer.param_groups[0]['lr'] < 1e-6:
        break


In [ ]:
model=torch.load("model/CNN_Transformer240.1402118945944.pth")
# model=torch.load("model/CNN_BiGRU_Attention238.75645760832163.pth")
#计算测试集MSE
test_data_dir="data/test_data.xlsx"
test_data_loader = Test_Loader(test_data_dir, train_data_loader.scaler)
test_data = DataLoader(test_data_loader, batch_size=1, shuffle=False)
print(len(test_data))

In [ ]:
# for features, targets in test_data:
#     features = features.cuda()
#     targets = targets.cuda()
#     test_model = model.cuda()
#     outputs = test_model(features)
#     print(outputs.cpu().detach().numpy()[0][0][0])
#     break


In [ ]:
#计算测试集上的MSE,RMSE,MAE,R2
def test(test_model, test_data):
    test_model.eval()

    YS_true = []
    YS_pred = []



    for features, targets in test_data:
        features = features.cuda()
        targets = targets.cuda()
        test_model = test_model.cuda()
        outputs = test_model(features)
        YS_true.append(targets.cpu().detach().numpy()[0][0][0])
        YS_pred.append(outputs.cpu().detach().numpy()[0][0][0])

    #计算YS的RMSE,MAPE,R2
    YS_true = np.array(YS_true)
    YS_pred = np.array(YS_pred)
    YS_MSE = mean_squared_error(YS_true, YS_pred)
    YS_RMSE = np.sqrt(YS_MSE)
    YS_MAPE = np.mean(np.abs((YS_pred - YS_true) / YS_true))
    YS_R2 = 1 - YS_MSE / np.var(YS_true)
    print(f'YS_RMSE: {YS_RMSE:.4f},YS_MAPE: {YS_MAPE:.4f},YS_R2: {YS_R2:.4f}')
    
    return YS_pred,  YS_true,

YS_pred, YS_true = test(model, test_data)